# Spend Analysis Workflow
Step-by-step spend analysis, segmentation, dashboarding, and export.

## Step 1: Import, preprocess & load data

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from source.data_processing.analysis_utils import (
    preprocess_detailed_data, make_spend_col,
    compute_abc_tiers,
    analyze_spend_distribution, analyze_supplier_distribution,
    abc_segmentation_analysis, export_results_per_class3
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# 1. Load raw data from BigQuery
from source.data_processing.analysis_utils import fetch_purchase_data_enriched
from source.db_connect.bigquery_connector import BigQueryConnector

bq = BigQueryConnector()
df_raw = fetch_purchase_data_enriched(bq, limit=None)  # or None for all
print('Raw data shape:', df_raw.shape)

# 2. Preprocess data (no spend column logic, just use 'Amount Eur')
from source.data_processing.analysis_utils import preprocess_detailed_data

df = preprocess_detailed_data(df_raw)
print('Columns after preprocess_detailed_data:', df.columns.tolist())



Initializing BigQuery connector...
   Project ID: kramp-sharedmasterdata-prd
   Credentials path: C:\Users\madsh\AppData\Roaming\gcloud\application_default_credentials.json
   Initializing client...
   Checking credentials path...
   Credentials file found; using explicit credentials
   Loading credentials file...
   Credentials file loaded; analyzing format
   Detected user account credentials
   Refreshing expired credentials...
   Creating BigQuery client...
   Client created successfully
Connected to BigQuery project: kramp-sharedmasterdata-prd
Fetching purchase data from BigQuery...
Executing query...
   Query preview: 
      SELECT * FROM kramp-sharedmasterdata-prd.MadsH.purchase_data_enriched
      
   Creating BigQuery client...
   Client created successfully
Connected to BigQuery project: kramp-sharedmasterdata-prd
Fetching purchase data from BigQuery...
Executing query...
   Query preview: 
      SELECT * FROM kramp-sharedmasterdata-prd.MadsH.purchase_data_enriched
      
   

In [ ]:
# Compute ABC tiers and summarize using 'Amount Eur'
from source.data_processing.analysis_utils import compute_abc_tiers

df_abc = compute_abc_tiers(df, spend_col='Amount Eur', thresholds=(0.80, 0.95), tier_col='abc_tier')



### Export to Excel

In [16]:
# Export segmentation results per Class3 using robust utility function
from source.data_processing.analysis_utils import export_results_per_class3
os.makedirs('exports_per_class3', exist_ok=True)
export_results_per_class3(df_abc, output_path="exports_per_class3/segmentation_per_class3.xlsx")

KeyError: 'Class3'

In [ ]:
# Diagnostic: Check for Class3 column and synthesize if missing
print('Columns in df_abc:', df_abc.columns.tolist())
if not any(col.lower().replace(' ', '').replace('_', '') == 'class3' for col in df_abc.columns):
    print('Class3 column not found. Attempting to synthesize using resolve_class3...')
    from source.data_processing.normalization_utils import resolve_class3
    df_abc['Class3'] = resolve_class3(df_abc)
    print('Class3 column synthesized. Columns now:', df_abc.columns.tolist())
else:
    print('Class3 column is present.')